### Assignment: Neighborhoods in Toronto

#### Importing libraries and reading dataset

In [2]:
import pandas as pd
import numpy as np

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)

In [3]:
TorontoN=tables[0].iloc[1:]

#### Lets have a look at the start of the table

In [4]:
TorontoN.head()

,Postal code,Borough,Neighborhood
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights


#### Deleting/Ignoring rows with borough not assigned and replacing "/" with "," as done in first part of assignment

In [5]:
TorontoN=TorontoN[TorontoN['Borough']!="Not assigned"]
TorontoN['Neighborhood']=TorontoN['Neighborhood'].str.replace("/", ",")
TorontoN.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


#### Adding and initializing columns Latitude and Longitude

In [6]:
TorontoN['Latitude']=0.0
TorontoN['Longitude']=0.0
TorontoN.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,0.0,0.0
3,M4A,North York,Victoria Village,0.0,0.0
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",0.0,0.0
5,M6A,North York,"Lawrence Manor , Lawrence Heights",0.0,0.0
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",0.0,0.0


#### Tried using geocoder library to get the lattitude and longitude but it always return "None" and go on infinite loop as shown below. So I had to go back to reading csv provided

In [15]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="ny_explorer")

i=0
for postal_code in TorontoN['Postal code']:
    # loop until you get the coordinates
    g = None
    while(g is None):
        g = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
        print('{}, Toronto, Ontario'.format(postal_code))
        
    TorontoN['Latitude'].iloc[i] = g.latitude
    TorontoN['Longitude'].iloc[i] = g.longitude
    i=i+1

print (i)
TorontoN.head()

M3A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario
M4A, Toronto, Ontario


KeyboardInterrupt: 

In [16]:
TorontoN.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.653482,-79.383935
3,M4A,North York,Victoria Village,0.000000,0.000000
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",0.000000,0.000000
5,M6A,North York,"Lawrence Manor , Lawrence Heights",0.000000,0.000000
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",0.000000,0.000000


#### Read the csv provided and save in another dataframe

In [17]:
Toronto_Loc = pd.read_csv('http://cocl.us/Geospatial_data')
Toronto_Loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
Toronto_Loc.shape

(103, 3)

#### Noticed that the column "Postal Code" has capital C in this file, so make the column name same as original dataframe, which can be then easy to merge. Also colums Latitude and Longitude were dropped from original dataframe as previous method didnt work

In [19]:
Toronto_Loc.rename(columns={'Postal Code':'Postal code'},inplace=True)
Toronto_Loc.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
TorontoN.drop(columns=['Latitude','Longitude'], inplace=True)
TorontoN.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


#### Merging the two dataframe to have dataframe required in assignment. Thank you!!!

In [23]:
TorontoN=pd.merge(TorontoN, Toronto_Loc, on="Postal code")

In [26]:
TorontoN.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


### Start of map visualization using the dataframe of Toronto

#### Printed how many boroughs are present in the dataframe and plot it at zoom level of 11.

In [27]:
print('The dataframe has {} boroughs and {} postcodes.'.format(len(TorontoN['Borough'].unique()), TorontoN.shape[0]))

The dataframe has 10 boroughs and 103 postcodes.


In [28]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(TorontoN['Latitude'], TorontoN['Longitude'], TorontoN['Borough'], TorontoN['Neighborhood'], TorontoN['Postal code']):
    label = '{}, {}, {}'.format(postcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Looking at the Toronto neighborhood the postcodes are densely situated south of Toronto whereas it is scatted more as you go outskirt of Toronto. 

#### Next I filter and select the Neighborhoods which has "Toronto" name in the Borough

In [40]:
TorontoT=TorontoN[TorontoN['Borough'].str.contains('Toronto')]
TorontoT.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Filtered data is visualized again and one can observe that most of these post codes are present south of Toronto

In [42]:
map_toronto_borough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(TorontoT['Latitude'], TorontoT['Longitude'], TorontoT['Borough'], TorontoT['Neighborhood'], TorontoT['Postal code']):
    label = '{}, {}, {}'.format(postcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_borough)  
    
map_toronto_borough